In [1]:
!pip install pmaw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from pmaw import PushshiftAPI
import datetime as dt

In [3]:
# Extração local no subreddit do país

start_time = int(dt.datetime(2020, 1, 1).timestamp())  
end_time = int(dt.datetime(2020, 7, 31).timestamp())   

api = PushshiftAPI()
posts = api.search_submissions(subreddit='ecuador',
                               after = start_time, 
                               before = end_time, 
                               q = '"coronavirus"||"covid"||"covid19"||"covid-19"||"corona virus"||"sars cov"||"sarscov"||"sars-cov-2"',
                               limit = None,
                               fields = ['id', 'author', 'created_utc', 'domain', 'url', 'title', 'num_comments', 'full_link', 'permalink'])
post_list_local = [post for post in posts]

In [4]:
# Extração geral no subreddit Coronavirus

start_time = int(dt.datetime(2020, 1, 1).timestamp())  
end_time = int(dt.datetime(2020, 7, 31).timestamp())   

api = PushshiftAPI()
posts = api.search_submissions(subreddit='Coronavirus',
                               after = start_time, 
                               before = end_time, 
                               q = 'ecuador',
                               limit = None,
                               fields = ['id', 'author', 'created_utc', 'domain', 'url', 'title', 'num_comments', 'full_link', 'permalink'])
post_list_global = [post for post in posts]

In [5]:
from time import sleep
import requests
import pandas as pd

df_post = pd.json_normalize(post_list_local)
print(df_post['id'].shape)

comments = []
i = 1
for post in df_post['id'].values:
  r = requests.get(f"https://api.pushshift.io/reddit/comment/search?q=*&link_id={post}&fields=author,body,created_utc,id,parent_id&limit=1000")
  while r.status_code != 200:
    sleep(min((2**i), 60))
    r = requests.get(f"https://api.pushshift.io/reddit/comment/search?q=*&link_id={post}&fields=author,body,created_utc,id,parent_id&limit=1000")
    i += 1

  i = 1
  comments.append(r.json())

pd.json_normalize(pd.json_normalize(comments).explode('data')['data']).to_csv('ecuador_local.csv')

(75,)


In [6]:
from time import sleep
import requests
import pandas as pd

df_post = pd.json_normalize(post_list_global)
print(df_post['id'].shape)

comments = []
i = 1
for post in df_post['id'].values:
  r = requests.get(f"https://api.pushshift.io/reddit/comment/search?q=*&link_id={post}&fields=author,body,created_utc,id,parent_id&limit=1000")
  while r.status_code != 200:
    sleep(min((2**i), 60))
    r = requests.get(f"https://api.pushshift.io/reddit/comment/search?q=*&link_id={post}&fields=author,body,created_utc,id,parent_id&limit=1000")
    i += 1

  i = 1
  comments.append(r.json())

pd.json_normalize(pd.json_normalize(comments).explode('data')['data']).to_csv('ecuador_global.csv')

(156,)


In [8]:
pd.read_csv('/content/ecuador_global.csv')

,Unnamed: 0,author,body,created_utc,id,parent_id
0,0,KingSnazz32,*El* [***Ministerio de Salud Pública***](http...,1.583235e+09,fjcs83n,t3_fct5pl
1,1,AutoModerator,Welcome to r/Coronavirus! We have a very speci...,1.583234e+09,fjcs5ft,t3_fct5pl
2,2,BaKe-NeKoLoLi,That's 2 more than she should but true,1.583329e+09,fjgku34,t1_fjck5p6
3,3,CthulhuStankBreath,2? 😝,1.583224e+09,fjck5p6,t1_fjch9p4
4,4,JKR44,She came to Prague with American woman that fe...,1.583221e+09,fjcigw8,t1_fjchnc3
...,...,...,...,...,...,...
1452,1452,babydolleffie,Uhhhhhhhh,1.594440e+09,fxlpl9f,t3_hp2nqj
1453,1453,Lief1s600d,This is what worries me about my food deliveries.,1.594440e+09,fxlphst,t3_hp2nqj
1454,1454,YourWebcam,Your submission has been removed because\n\n\n...,1.594401e+09,fxjs0ht,t3_hokrfh
1455,1455,lzghome,"This afternoon, China's State Council Joint Pr...",1.594367e+09,fxidpys,t3_hokrfh


In [9]:
pd.read_csv('/content/ecuador_local.csv')

,Unnamed: 0,author,body,created_utc,id,parent_id
0,0,Laikusi,Por favor no cometais los mismos errores que e...,1.586777e+09,fn9lege,t3_fct4j9
1,1,strat1227,Toque de queda desde 2pm empieza mañana....\n\...,1.585103e+09,flfc29f,t3_fct4j9
2,2,RandomUserKnowledge,Let me know if I can help. Sent me a DM and I ...,1.584437e+09,fkq9ltu,t1_fkp2682
3,3,Edjoerv,"A partir de las 23:59 de hoy, 16 de marzo, ya ...",1.584419e+09,fkpt5us,t1_fkf624h
4,4,TheChaosPaladin,We should get a bot to do this for us but unfo...,1.584400e+09,fkp2682,t1_fkndxrl
...,...,...,...,...,...,...
426,426,TheChaosPaladin,I would heavily advise against coming to Ecuad...,1.595438e+09,fyw23dn,t3_hvws5q
427,427,VojskaKralj,Yeah but still most people are doing it well i...,1.595437e+09,fyw0mww,t1_fyvyedq
428,428,spastorm20,The big issue here is people not wearing the p...,1.595436e+09,fyvyedq,t3_hvws5q
429,429,flykidsblue1,Word,1.595435e+09,fyvwyec,t1_fyvwjkn
